In [137]:
import torch
import os
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

In [138]:
#Data directory path
data_path = r"C:\Users\VirtualUser\Desktop\Abraham_Internship\Abraham\Data"
data_path

'C:\\Users\\VirtualUser\\Desktop\\Abraham_Internship\\Abraham\\Data'

In [139]:
#Train and validation directories path
train_dir = data_path + "\\train"
val_dir = data_path + "\\validation"
train_dir

'C:\\Users\\VirtualUser\\Desktop\\Abraham_Internship\\Abraham\\Data\\train'

In [140]:
#Listng the number of images in each folder
traincat = os.listdir(os.path.join(train_dir, "Cat"))
traindog = os.listdir(os.path.join(train_dir, "Dog"))
valcat = os.listdir(os.path.join(val_dir, "Cat"))
valdog = os.listdir(os.path.join(val_dir, "Dog"))
print("Train Cats:", len(traincat))
print("Train Dogs:", len(traindog))
print("Val Cats:", len(valcat))
print("Val Dogs:", len(valdog))

Train Cats: 9999
Train Dogs: 9999
Val Cats: 2500
Val Dogs: 2500


In [141]:
#Data preprocessing/augmentation
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)), #Resizing the images to 224x224
    transforms.RandomRotation(15), #Random rotation
    transforms.RandomHorizontalFlip(), #Random horizontal flip
    transforms.ToTensor(), #Converting the images to tensors
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2), #Color jitter that augments the brightness, contrast, saturation, and hue of the images
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), #Normalizing the images by subtracting the mean and dividing by the standard deviation
])

val_transforms = transforms.Compose([ #validaiton doesn't need data augmentation
    transforms.Resize((224, 224)), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [142]:
train_dataset = ImageFolder(train_dir, transform=train_transforms) #Creating the datasets for training and validation
val_dataset = ImageFolder(val_dir, transform=val_transforms)

In [143]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4) #Creating the dataloaders with the datasets and batch size of 32 and num_workers of 4 for parallel processing with shuffling to randomize the order of the images
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4) #Validation doesn't need shuffling cause it's not training
class_names = train_dataloader.dataset.classes
print(class_names)

['Cat', 'Dog']


In [144]:
print("No: of training images:", len(train_dataset))
print("No: of validation images:", len(val_dataset))
print("No: of training batches:", len(train_dataloader))
print("No: of validation batches:", len(val_dataloader))

No: of training images: 19998
No: of validation images: 5000
No: of training batches: 625
No: of validation batches: 157


In [145]:
i, j = next(iter(train_dataloader)) #Getting the first batch of images
print(i.shape, j.shape) #The shape is (32, 3, 224, 224) and (32,) which denotes the number of images in the batch, the number of channels, and the size of the images and no: of images-label pairs in the batch

torch.Size([32, 3, 224, 224]) torch.Size([32])


In [146]:
print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"GPU Name: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}") #Checking if GPU is available

CUDA Available: False
GPU Name: CPU


In [147]:
resnet50 = models.resnet50(pretrained=True) #Loading the pre-trained ResNet model with pretrained weights from torchvision 
print(resnet50)

c:\Users\VirtualUser\Desktop\Abraham_Internship\Abraham\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\VirtualUser\Desktop\Abraham_Internship\Abraham\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [148]:
#Freezing every layer of the ResNet model
for param in resnet50.parameters():
    param.requires_grad = False

In [149]:
resnet_features = nn.Sequential(*list(resnet50.children())[:-1])

#Testing the feature extraction layer
sample_input = torch.randn(2, 3, 224, 224)  # batch of 2 images
features = resnet_features(sample_input)
print(features.shape) 
print(features)

torch.Size([2, 2048, 1, 1])
tensor([[[[0.3696]],

         [[0.6529]],

         [[0.4742]],

         ...,

         [[0.1274]],

         [[0.5598]],

         [[0.2134]]],


        [[[0.3525]],

         [[0.3176]],

         [[0.4274]],

         ...,

         [[0.5178]],

         [[0.3307]],

         [[0.4098]]]])


In [158]:
custom_classifier = nn.Sequential(
    nn.Flatten(),
    nn.Linear(2048, 512),
    nn.ReLU(),
    nn.Dropout(0.30),
    nn.Linear(512, 2),
)

#Testing the custom classifier
prob = custom_classifier(features)
print(prob.shape)
print(prob)

torch.Size([2, 2])
tensor([[ 0.1155, -0.1555],
        [-0.1068, -0.0010]], grad_fn=<AddmmBackward0>)


In [159]:
def model_forward (x):
    features = resnet_features(x)
    output = custom_classifier(features)
    return output

#Testing forward function
sample_output = model_forward(sample_input)
print(sample_output.shape)
print(sample_output)

torch.Size([2, 2])
tensor([[-0.0152, -0.0511],
        [ 0.0494, -0.1888]], grad_fn=<AddmmBackward0>)


In [160]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(custom_classifier.parameters(), lr=0.001)

In [161]:
def train_one_epoch(model_forward,
                    classifier,
                    dataloader,
                    loss_function,
                    optimizer):

    # Put classifier in training mode
    classifier.train()

    # Variables to track performance
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    # Loop through batches
    for inputs, labels in dataloader:

        # Clear old gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model_forward(inputs)

        # Compute loss
        loss = loss_function(outputs, labels)

        # Backpropagation
        loss.backward()

        # Update weights
        optimizer.step()

        # Accumulate total loss
        total_loss += loss.item() * inputs.size(0)

        # Get predicted classes
        _, preds = torch.max(outputs, 1)

        # Count correct predictions
        correct_predictions += torch.sum(preds == labels).item()

        # Count total samples
        total_samples += labels.size(0)

    # Compute average loss
    average_loss = total_loss / total_samples

    # Compute accuracy percentage
    accuracy = (correct_predictions / total_samples) * 100

    return average_loss, accuracy

In [162]:
def validate_one_epoch(model_forward,
                       classifier,
                       dataloader,
                       loss_function):

    # Put classifier in evaluation mode
    classifier.eval()

    # Variables to track performance
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    # Disable gradient computation
    with torch.no_grad():

        # Loop through validation batches
        for inputs, labels in dataloader:


            # Forward pass
            outputs = model_forward(inputs)

            # Compute loss
            loss = loss_function(outputs, labels)

            # Accumulate total loss
            total_loss += loss.item() * inputs.size(0)

            # Get predicted classes
            _, preds = torch.max(outputs, 1)

            # Count correct predictions
            correct_predictions += torch.sum(preds == labels).item()

            # Count total samples
            total_samples += labels.size(0)

    # Compute average loss
    average_loss = total_loss / total_samples

    # Compute accuracy percentage
    accuracy = (correct_predictions / total_samples) * 100

    return average_loss, accuracy

In [ ]:
num_epochs = 1

for epoch in range(num_epochs):

    train_loss, train_acc = train_one_epoch(
        model_forward,
        custom_classifier,
        train_dataloader,
        criterion,
        optimizer,
    )

    val_loss, val_acc = validate_one_epoch(
        model_forward,
        custom_classifier,
        val_dataloader,
        criterion,
    )

    print("Epoch:", epoch + 1)
    print("Train Loss:", round(train_loss, 4), 
          "Train Accuracy:", round(train_acc, 2), "%")
    print("Val Loss:", round(val_loss, 4), 
          "Val Accuracy:", round(val_acc, 2), "%")
    print()